In [14]:
import pandas as pd
import requests

# Disable SSL warnings (not recommended for production)
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# API Endpoints
quiz_endpoint_url = "https://jsonkeeper.com/b/LLQT"
quiz_submission_url = "https://api.jsonserve.com/rJvd7g"
api_endpoint_url = "https://api.jsonserve.com/XgAgFJ"

# Fetch the data
quiz_data_response = requests.get(quiz_endpoint_url, verify=False)
quiz_submission_data_response = requests.get(quiz_submission_url, verify=False)
historical_data_response = requests.get(api_endpoint_url, verify=False)

# Convert the JSON data into Python objects
quiz_data = quiz_data_response.json()
quiz_submission_data = quiz_submission_data_response.json()
historical_data = historical_data_response.json()

# Normalize nested JSON structures
quiz_submission_data_flat = pd.json_normalize(quiz_submission_data)
quiz_data_df = pd.DataFrame(quiz_data)
historical_data_df = pd.DataFrame(historical_data)

# Display DataFrames
print("Quiz Data:")
print(quiz_data_df.head())

print("\nQuiz Submission Data:")
print(quiz_submission_data_flat.head())

print("\nHistorical Data:")
print(historical_data_df.head())


Quiz Data:
                                                               quiz
id                                                               43
name                                                           None
title             Structural Organisation in Animals and Plants (7)
description                                                        
difficulty_level                                               None

Quiz Submission Data:
       id  quiz_id                       user_id  \
0  336566       43  7ZXdz3zHuNcdg9agb5YpaOGLQqw2   

                    submitted_at                     created_at  \
0  2025-01-17T15:51:29.859+05:30  2025-01-17T15:51:29.871+05:30   

                      updated_at  score  trophy_level accuracy speed  ...  \
0  2025-01-17T15:51:29.871+05:30     32             2     80 %   100  ...   

  quiz.exam_id quiz.show_unanswered  quiz.ends_at  quiz.lives quiz.live_count  \
0         None                False    2025-01-18        None       Free Test   



In [12]:
import pandas as pd
import numpy as np

# Load your datasets (replace with the actual file paths or data sources)
# Example: quiz_data, submission_data, historical_data are DataFrames
quiz_data = pd.DataFrame({
    "id": [43],
    "title": ["Structural Organisation in Animals and Plants"],
    "questions_count": [128],
    "max_mistake_count": [15],
})

submission_data = pd.DataFrame({
    "id": [336566],
    "quiz_id": [43],
    "score": [32],
    "accuracy": ["80 %"],
    "speed": [100],
    "mistakes_corrected": [3],
    "initial_mistake_count": [12],
    "response_map": ["{'2523': 10109, '2529': 10130, '2533': 10149}"],
})

historical_data = pd.DataFrame({
    "id": [336497, 336448, 333330, 333242, 329504],
    "quiz_id": [51, 6, 51, 6, 51],
    "score": [108, 92, 116, 36, 36],
    "accuracy": ["90 %", "100 %", "96 %", "90 %", "31 %"],
    "mistakes_corrected": [9, 3, 11, 1, 0],
    "initial_mistake_count": [12, 3, 12, 2, 20],
})

# Convert percentages to numeric
submission_data['accuracy'] = submission_data['accuracy'].str.rstrip(' %').astype(float)
historical_data['accuracy'] = historical_data['accuracy'].str.rstrip(' %').astype(float)

# Analyze the Data
def analyze_data(quiz_data, submission_data, historical_data):
    # Group historical data by quiz ID
    historical_summary = historical_data.groupby('quiz_id').agg(
        avg_score=('score', 'mean'),
        avg_accuracy=('accuracy', 'mean'),
        avg_mistakes=('mistakes_corrected', 'mean'),
        attempts=('id', 'count')
    ).reset_index()

    # Merge with quiz data for context
    analysis = pd.merge(historical_summary, quiz_data, left_on='quiz_id', right_on='id', how='left')
    return analysis

# Generate Insights
def generate_insights(submission_data, historical_data):
    # Weak areas: Low accuracy, low score
    weak_topics = historical_data[historical_data['accuracy'] < 70].groupby('quiz_id').agg(
        avg_score=('score', 'mean'),
        avg_accuracy=('accuracy', 'mean')
    )

    # Improvement trends: Compare last attempt with mean performance
    historical_data['improvement'] = historical_data['score'] - historical_data['score'].mean()
    improvement_trends = historical_data[['quiz_id', 'score', 'improvement']]

    return weak_topics, improvement_trends

# Create Recommendations
def create_recommendations(weak_topics):
    recommendations = []
    for quiz_id in weak_topics.index:
        recommendations.append(f"Focus on quiz {quiz_id} with average accuracy {weak_topics.loc[quiz_id, 'avg_accuracy']}%.")
    return recommendations

# Execute functions
analysis = analyze_data(quiz_data, submission_data, historical_data)
weak_topics, improvement_trends = generate_insights(submission_data, historical_data)
recommendations = create_recommendations(weak_topics)

# Display results
print("Analysis Summary:\n", analysis)
print("\nWeak Topics:\n", weak_topics)
print("\nImprovement Trends:\n", improvement_trends)
print("\nRecommendations:\n", recommendations)


Analysis Summary:
    quiz_id  avg_score  avg_accuracy  avg_mistakes  attempts  id title  \
0        6  64.000000     95.000000      2.000000         2 NaN   NaN   
1       51  86.666667     72.333333      6.666667         3 NaN   NaN   

   questions_count  max_mistake_count  
0              NaN                NaN  
1              NaN                NaN  

Weak Topics:
          avg_score  avg_accuracy
quiz_id                         
51            36.0          31.0

Improvement Trends:
    quiz_id  score  improvement
0       51    108         30.4
1        6     92         14.4
2       51    116         38.4
3        6     36        -41.6
4       51     36        -41.6

Recommendations:
 ['Focus on quiz 51 with average accuracy 31.0%.']
